In [68]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import unicodedata

#### date, time column 이름 통일

In [ ]:
path = '../../data/weather_forecast_data/2015_2021_prediction_total_data'
files = os.listdir(path)
files = [f for f in files if f.endswith('.parquet')]
for f in files:
    file = pd.read_parquet(os.path.join(path, f))

    # day 컬럼 변환: 정수 → 문자열 → 날짜 포맷
    file['day'] = pd.to_datetime(file['day'].astype(str), format='%Y%m%d').dt.strftime('%Y-%m-%d')
    file.rename(columns={'day': 'date'}, inplace=True)

    # hour 컬럼 변환 → time 컬럼으로 이름 변경
    file['time'] = file['hour'].astype(str).str.zfill(4).str.replace(r'(\d{2})(\d{2})', r'\1:\2', regex=True)
    file.drop(columns='hour', inplace=True)  # 기존 hour 컬럼 제거

    # 다시 저장
    file.to_parquet(os.path.join(path, f), index=False)
    #print(f,file['hour'].unique())

In [2]:
path = '../../data/weather_forecast_data/total_data2'
files = os.listdir(path)
files = [f for f in files if f.endswith('.parquet')]
for f in files:
    file = pd.read_parquet(os.path.join(path, f))

    # day 컬럼 변환: 정수 → 문자열 → 날짜 포맷
    file['day'] = pd.to_datetime(file['day'].astype(str), format='%Y%m%d').dt.strftime('%Y-%m-%d')
    file.rename(columns={'day': 'date'}, inplace=True)

    # hour 컬럼 변환 → time 컬럼으로 이름 변경
    file['time'] = file['hour'].astype(str).str.zfill(4).str.replace(r'(\d{2})(\d{2})', r'\1:\2', regex=True)
    file.drop(columns='hour', inplace=True)  # 기존 hour 컬럼 제거

    # 다시 저장
    file.to_parquet(os.path.join(path, f), index=False)
    #print(f,file['hour'].unique())

In [7]:
# 신데이터 합치기
prev_path = '../../data/weather_forecast_data/2015_2021_prediction_total_data'
path = '../../data/weather_forecast_data/total_data2'
files = os.listdir(path)
files = [f for f in files if f.endswith('.parquet')]

# 병합 기준 컬럼
merge_keys = ['date', 'time', 'forecast']

for f in files:
    prev = pd.read_parquet(os.path.join(prev_path, f))
    file = pd.read_parquet(os.path.join(path, f))

    merge = pd.concat([file, prev]).drop_duplicates(subset=merge_keys, keep='first').reset_index(drop=True)


In [46]:
# 완전한 인코딩 + 데이터타입 처리
import pandas as pd
import os
import unicodedata

def normalize_columns(df):
    """한글 컬럼명 정규화"""
    df.columns = [unicodedata.normalize('NFC', col) for col in df.columns]
    return df

prev_path = '../../data/weather_forecast_data/2015_2021_prediction_total_data'
out_path = '../../data/weather_forecast_data/final_total_data'
os.makedirs(out_path,exist_ok=True)
path = '../../data/weather_forecast_data/total_data2'
files = os.listdir(path)
files = [f for f in files if f.endswith('.parquet')]

merge_keys = ['date', 'time', 'forecast']

for f in files:
    print(f"Processing file: {f}")
    
    prev = pd.read_parquet(os.path.join(prev_path, f))
    file = pd.read_parquet(os.path.join(path, f))
    
    # 1. 컬럼명 정규화 (인코딩 문제 해결)
    prev = normalize_columns(prev)
    file = normalize_columns(file)
    
    # 2. 1시간기온 -> 3시간기온 변경
    if '1시간기온' in file.columns:
        file = file.rename(columns={'1시간기온': '3시간기온'})
    
    # 3. 데이터 타입 정리
    # date를 datetime으로 변환
    for df in [prev, file]:
        if df['date'].dtype == 'object':
            df['date'] = pd.to_datetime(df['date'])
    
    # time을 시간 순서로 정렬하기 위해 시간 값 추출
    for df in [prev, file]:
        if df['time'].dtype == 'object':
            # '12:00' -> 12 형태로 변환 (시간 값만)
            df['time_numeric'] = df['time'].str.split(':').str[0].astype(int)
    
    # forecast는 숫자형으로 변환
    for df in [prev, file]:
        if df['forecast'].dtype == 'object':
            df['forecast'] = pd.to_numeric(df['forecast'], errors='coerce')
    
    # 4. 겹치는 기간 처리 (2021-06-29 기준)
    overlap_start = pd.to_datetime('2021-06-29')
    
    prev_filtered = prev[prev['date'] < overlap_start]
    file_filtered = file[file['date'] >= overlap_start]
    
    print(f"  prev 원본: {len(prev)} rows")
    print(f"  file 원본: {len(file)} rows")
    print(f"  prev 필터링 후: {len(prev_filtered)} rows (< {overlap_start.date()})")
    print(f"  file 필터링 후: {len(file_filtered)} rows (>= {overlap_start.date()})")
    
    # 5. 합치기
    combined = pd.concat([prev_filtered, file_filtered], ignore_index=True)
    
    # 6. 혹시 모를 중복 제거
    before_dedup = len(combined)
    combined = combined.drop_duplicates(subset=merge_keys, keep='last')
    after_dedup = len(combined)
    
    if before_dedup != after_dedup:
        print(f"  중복 제거: {before_dedup} -> {after_dedup} ({before_dedup - after_dedup} 개 제거)")
    
    # 7. 정렬 (date, time_numeric, forecast 순으로 정렬)
    combined = combined.sort_values(by=['date', 'time_numeric', 'forecast']).reset_index(drop=True)
    
    # 8. time_numeric 컬럼 제거 (필요 없어짐)
    combined = combined.drop('time_numeric', axis=1)
    
    # 9. 결과 저장
    output_path = os.path.join(out_path, f)
    combined.to_parquet(output_path, index=False)
    
    print(f"  최종 합쳐진 데이터: {len(combined)} rows")
    print(f"  저장 완료: {output_path}\n")

print("모든 파일 처리 완료!")

Processing file: 이천_prediction_weather_data.parquet
  prev 원본: 350211 rows
  file 원본: 280094 rows
  prev 필터링 후: 349407 rows (< 2021-06-29)
  file 필터링 후: 280094 rows (>= 2021-06-29)
  중복 제거: 629501 -> 628993 (508 개 제거)
  최종 합쳐진 데이터: 628993 rows
  저장 완료: ../../data/weather_forecast_data/final_total_data/이천_prediction_weather_data.parquet

Processing file: 동해_prediction_weather_data.parquet
  prev 원본: 350211 rows
  file 원본: 279586 rows
  prev 필터링 후: 349407 rows (< 2021-06-29)
  file 필터링 후: 279586 rows (>= 2021-06-29)
  최종 합쳐진 데이터: 628993 rows
  저장 완료: ../../data/weather_forecast_data/final_total_data/동해_prediction_weather_data.parquet

Processing file: 서귀포_prediction_weather_data.parquet
  prev 원본: 350211 rows
  file 원본: 279586 rows
  prev 필터링 후: 349407 rows (< 2021-06-29)
  file 필터링 후: 279586 rows (>= 2021-06-29)
  최종 합쳐진 데이터: 628993 rows
  저장 완료: ../../data/weather_forecast_data/final_total_data/서귀포_prediction_weather_data.parquet

Processing file: 충주_prediction_weather_data.parquet
  p

In [70]:
import shutil 
out_path = '../../data/weather_forecast_data/final_total_data'

prev_files = os.listdir(prev_path)
prev_files.remove('.DS_Store')

ff_processed = {unicodedata.normalize('NFC', f.split('_')[0]).strip() for f in files}

for f_prev in prev_files:
    # prev_files에서 추출한 이름도 동일하게 정규화하고 공백 제거합니다.
    processed_name_from_prev = unicodedata.normalize('NFC', f_prev.split('_')[0]).strip()

    if processed_name_from_prev not in ff_processed:
        source_file_path = os.path.join(prev_path, f_prev)
        destination_file_path = os.path.join(out_path, f_prev)

        try:
            shutil.copy2(source_file_path, destination_file_path)
            print(f"'{f_prev}' 파일을 '{out_path}'로 복사했습니다.")
        except FileNotFoundError:
            print(f"오류: 원본 파일 '{source_file_path}'을(를) 찾을 수 없습니다.")
        except Exception as e:
            print(f"오류: '{f_prev}' 파일 복사 중 예외 발생: {e}")

'제주_prediction_weather_data.parquet' 파일을 '../../data/weather_forecast_data/final_total_data'로 복사했습니다.
'영월_prediction_weather_data.parquet' 파일을 '../../data/weather_forecast_data/final_total_data'로 복사했습니다.
'인천_prediction_weather_data.parquet' 파일을 '../../data/weather_forecast_data/final_total_data'로 복사했습니다.
'성산일출_prediction_weather_data.parquet' 파일을 '../../data/weather_forecast_data/final_total_data'로 복사했습니다.
'부산_prediction_weather_data.parquet' 파일을 '../../data/weather_forecast_data/final_total_data'로 복사했습니다.


#### date,time,forecast column 형식 수정

In [47]:

def preprocess_forecast_data(forecast_path, filename, output_prefix):
    """
    날씨 예보 데이터를 전처리하여 '전날14시에예측'과 '전날20시에예측' 파일 생성
    
    Args:
        forecast_path: 원본 forecast 데이터 파일 경로
        filename: 지역이름 (ex. 전주)
        output_prefix: 출력 파일 경로 접두사
    
    Returns:
        None (파일 저장)
    """
    print(f"Loading forecast data from {forecast_path}...")
    # 예보 데이터 로드
    forecast_df = pd.read_parquet(forecast_path)
    
    # 데이터 타입 확인 및 변환
    if forecast_df['date'].dtype == 'object':
        forecast_df['date'] = pd.to_datetime(forecast_df['date'])
    
    # time 열이 문자열인지 확인하고 표준 형식으로 변환
    if forecast_df['time'].dtype == 'object':
        # time이 'HH:MM' 형식인 경우 처리
        try:
            # 시간 형식 통일
            forecast_df['time'] = forecast_df['time'].apply(
                lambda x: f"{int(x.split(':')[0]):02d}:00" if ':' in str(x) else f"{int(x):02d}:00"
            )
        except:
            print("Warning: time column format conversion failed, assuming it's already in correct format")
    
    # 필요한 컬럼 목록 (원본 데이터에 맞게 조정 필요)
    forecast_columns = ['3시간기온', '풍향', '하늘상태', '풍속', '강수확률']
    available_columns = [col for col in forecast_columns if col in forecast_df.columns]
    
    if not available_columns:
        print("Warning: No forecast columns found in the data. Using all available columns.")
        # 사용 가능한 모든 컬럼 사용 (date, time, forecast 제외)
        available_columns = [col for col in forecast_df.columns if col not in ['date', 'time', 'forecast']]
    
    print(f"Using forecast columns: {available_columns}")
    
    # 데이터 타입 미리 확인 및 변환 시도
    for col in available_columns:
        try:
            # 데이터 타입 확인
            dtype = forecast_df[col].dtype
            print(f"Column {col} has dtype: {dtype}")
            
            # 문자열이나 object 타입이고 숫자로 변환 가능한지 시도
            if dtype == 'object' or dtype == 'string':
                # 'None' 문자열 처리
                forecast_df[col] = forecast_df[col].replace('None', np.nan)
                # 변환 시도
                forecast_df[col] = pd.to_numeric(forecast_df[col], errors='coerce')
                print(f"Converted column {col} to numeric. New dtype: {forecast_df[col].dtype}")
        except Exception as e:
            print(f"Warning: Error checking/converting column {col}: {e}")
    
    # 범주형 변수 식별 (하늘상태)
    categorical_cols = ['하늘상태']
    categorical_cols = [col for col in categorical_cols if col in available_columns]
    numerical_cols = [col for col in available_columns if col not in categorical_cols]
    
    print(f"Categorical columns: {categorical_cols}")
    print(f"Numerical columns: {numerical_cols}")
    
    # 범주형 변수의 데이터 타입 확인 (object로 변환 필요)
    for col in categorical_cols:
        if col in forecast_df.columns:
            forecast_df[col] = forecast_df[col].astype(str)
    
    # 두 가지 예측 시간 처리 (14:00, 20:00)
    for predict_time in ["14:00", "20:00"]:
        print(f"\nProcessing {predict_time} forecasts...")
        # 해당 시간의 예보 데이터만 필터링
        time_forecasts = forecast_df[forecast_df['time'] == predict_time].copy()
        
        if len(time_forecasts) == 0:
            print(f"Warning: No data found for time {predict_time}. Skipping.")
            continue
        
        # 결과 데이터프레임 초기화
        result_data = []
        
        # 각 날짜별 처리
        for date in time_forecasts['date'].unique():
            # 해당 날짜의 예보 데이터
            date_forecasts = time_forecasts[time_forecasts['date'] == date]
            
            # numpy.datetime64를 pandas Timestamp로 변환하여 날짜 연산
            # 다음날 계산
            if isinstance(date, np.datetime64):
                # numpy.datetime64를 pandas Timestamp로 변환
                pd_date = pd.Timestamp(date)
                target_date = pd_date + pd.Timedelta(days=1)
            else:
                # 이미 pandas Timestamp 또는 datetime인 경우
                target_date = date + timedelta(days=1)
            
            # 각 시간대별 처리
            for hour in range(24):
                target_time = f"{hour:02d}:00"
                
                # 필요한 forecast 값 계산 (전날 14시 또는 20시부터 해당 시간까지의 시간 차이)
                predict_hour = int(predict_time.split(':')[0])
                hours_ahead = hour + (24 - predict_hour)
                
                # 정확히 일치하는 forecast 찾기
                matching_forecasts = date_forecasts[date_forecasts['forecast'] == hours_ahead]
                
                # 정확히 일치하는 forecast가 있으면 해당 데이터 사용
                if len(matching_forecasts) > 0:
                    forecast_values = matching_forecasts[available_columns].iloc[0].to_dict()
                    
                    # 결과 행 추가
                    result_row = {
                        'date': target_date,
                        'time': target_time,
                        'forecast_date': date,  # 원본 예보 날짜
                        'forecast_time': predict_time,  # 원본 예보 시간
                        'hours_ahead': hours_ahead,  # 예보 시점부터 몇 시간 후인지
                        'exact_match': True  # 정확히 일치하는 forecast 여부
                    }
                    result_row.update(forecast_values)
                    
                    result_data.append(result_row)
                else:
                    # 일치하는 forecast가 없는 경우 보간을 위해 빈 데이터 추가
                    result_row = {
                        'date': target_date,
                        'time': target_time,
                        'forecast_date': date,  # 원본 예보 날짜
                        'forecast_time': predict_time,  # 원본 예보 시간
                        'hours_ahead': hours_ahead,  # 예보 시점부터 몇 시간 후인지
                        'exact_match': False  # 정확히 일치하는 forecast 여부
                    }
                    # 컬럼은 있지만 값은 NaN으로 설정
                    for col in available_columns:
                        result_row[col] = np.nan
                    
                    result_data.append(result_row)
        
        # 결과 데이터프레임 생성
        result_df = pd.DataFrame(result_data)
        
        # 시간대별 정렬
        result_df = result_df.sort_values(['date', 'time'])
        
        # 보간 처리
        if len(result_df) > 0:
            # 시간을 숫자로 변환
            result_df['hour'] = result_df['time'].apply(lambda x: int(x.split(':')[0]))
            
            # 각 날짜별로 처리
            interpolated_data = []
            
            for date in result_df['date'].unique():
                date_data = result_df[result_df['date'] == date].copy()
                
                # 날짜별 데이터가 충분한지 확인
                if len(date_data) < 2:
                    print(f"Warning: Not enough data for interpolation on {date}. Skipping.")
                    continue
                
                # 시간별 정렬
                date_data = date_data.sort_values('hour')
                
                # 1. 수치형 변수 - 선형 보간 후 정수로 반올림
                if numerical_cols:
                    # 먼저 수치형 컬럼을 명시적으로 float 타입으로 변환
                    for col in numerical_cols:
                        try:
                            # 'None' 문자열 처리 (필요한 경우)
                            date_data[col] = date_data[col].replace('None', np.nan)
                            # 숫자형으로 변환 시도
                            date_data[col] = pd.to_numeric(date_data[col], errors='coerce')
                        except Exception as e:
                            print(f"Warning: Failed to convert column {col} to numeric: {e}")
                            # 변환 실패 시 해당 컬럼을 수치형 목록에서 제외
                            numerical_cols.remove(col)
                    
                    # 변환 후 수치형 컬럼이 남아 있는지 확인
                    if numerical_cols:
                        # 선형 보간 수행
                        date_data[numerical_cols] = date_data[numerical_cols].interpolate(method='linear')
                        
                        # 정수 값이 필요한 컬럼 반올림
                        for col in numerical_cols:
                            # NaN 값이 아닌 경우에만 반올림
                            if not date_data[col].isna().all():
                                # 소수점이 있는지 확인 (정수 값이면 그대로 유지)
                                if (date_data[col] % 1 != 0).any():
                                    date_data[col] = date_data[col].round().astype('Int64')
                
                # 2. 범주형 변수 - 전방 채우기 (forward fill)
                if categorical_cols:
                    # 각 범주형 변수별로 처리
                    for col in categorical_cols:
                        # NaN 값이 있는 경우에만 처리
                        if date_data[col].isna().any():
                            # 전방 채우기 (앞의 값으로 채움)
                            date_data[col] = date_data[col].fillna(method='ffill')
                            # 후방 채우기 (뒤의 값으로 채움, 앞부분에 NaN이 있는 경우)
                            date_data[col] = date_data[col].fillna(method='bfill')
                
                # 결과에 추가
                interpolated_data.append(date_data)
            
            # 보간된 데이터 결합
            if interpolated_data:
                result_df = pd.concat(interpolated_data).drop(columns=['hour'])
            else:
                print(f"Warning: No data after interpolation for {predict_time}")
                continue
        
        # 파일 저장
        output_path = os.path.join(output_prefix,f'{predict_time[:2]}00',f'{filename}.parquet')
        result_df.to_parquet(output_path, index=False)
        print(f"Saved preprocessed forecast data to {output_path}")
        print(f"Created {len(result_df)} records for {len(result_df['date'].unique())} days")


path = '../../data/weather_forecast_data/final_total_data'
files = os.listdir(path)
files = [f for f in files if f.endswith('.parquet')]
for f in files:
    forecast_path = os.path.join(path, f)  # 원본 forecast 파일 경로
    output_prefix = "../../data/weather_forecast_data/nextday_forecast"  # 출력 파일 경로 접두사

    # 전처리 실행
    preprocess_forecast_data(forecast_path,f.split('_')[0], output_prefix)


Loading forecast data from ../../data/weather_forecast_data/final_total_data/이천_prediction_weather_data.parquet...
Using forecast columns: ['3시간기온', '풍향', '하늘상태', '풍속', '강수확률']
Column 3시간기온 has dtype: object
Converted column 3시간기온 to numeric. New dtype: float64
Column 풍향 has dtype: object
Converted column 풍향 to numeric. New dtype: float64
Column 하늘상태 has dtype: object
Converted column 하늘상태 to numeric. New dtype: float64
Column 풍속 has dtype: object
Converted column 풍속 to numeric. New dtype: float64
Column 강수확률 has dtype: object
Converted column 강수확률 to numeric. New dtype: float64
Categorical columns: ['하늘상태']
Numerical columns: ['3시간기온', '풍향', '풍속', '강수확률']

Processing 14:00 forecasts...
Saved preprocessed forecast data to ../../data/weather_forecast_data/nextday_forecast/1400/이천.parquet
Created 69864 records for 2911 days

Processing 20:00 forecasts...
Saved preprocessed forecast data to ../../data/weather_forecast_data/nextday_forecast/2000/이천.parquet
Created 69888 records for 2912 da

In [56]:
a = pd.read_parquet('/Users/soomin/Desktop/공공데이터/renewable-power-prediction/data/weather_forecast_data/final_total_data/동해_prediction_weather_data.parquet')
a

,date,forecast,3시간기온,강수확률,하늘상태,풍속,풍향,time
0,2015-01-01,4,-0.600000,0.000000,1.000000,5.300000,308.000000,02:00
1,2015-01-01,7,-2.400000,0.000000,1.000000,5.800000,286.000000,02:00
2,2015-01-01,10,-3.600000,0.000000,1.000000,3.200000,255.000000,02:00
3,2015-01-01,13,-4.600000,0.000000,1.000000,4.200000,220.000000,02:00
4,2015-01-01,16,-5.500000,10.000000,2.000000,4.600000,238.000000,02:00
...,...,...,...,...,...,...,...,...
628988,2022-12-31,60,-2.0,0.0,1.0,0.0,253.0,23:00
628989,2022-12-31,61,-2.0,0.0,1.0,0.0,254.0,23:00
628990,2022-12-31,62,-3.0,0.0,1.0,0.0,253.0,23:00
628991,2022-12-31,63,-3.0,0.0,1.0,0.0,253.0,23:00


In [73]:
a = pd.read_parquet('/Users/soomin/Desktop/공공데이터/renewable-power-prediction/data/weather_forecast_data/nextday_forecast/2000/동해.parquet')
a

,date,time,forecast_date,forecast_time,hours_ahead,exact_match,3시간기온,풍향,하늘상태,풍속,강수확률
0,2015-01-02,00:00,2015-01-01,20:00,4,True,-4.0,276.0,3.0,4.0,20.0
1,2015-01-02,01:00,2015-01-01,20:00,5,False,-3.0,289.0,3.0,5.0,17.0
2,2015-01-02,02:00,2015-01-01,20:00,6,False,-2.0,303.0,3.0,6.0,13.0
3,2015-01-02,03:00,2015-01-01,20:00,7,True,-1.0,316.0,2.0,7.0,10.0
4,2015-01-02,04:00,2015-01-01,20:00,8,False,-1.0,321.0,2.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...
69883,2023-01-01,19:00,2022-12-31,20:00,23,True,-4.0,256.0,1.0,0.0,0.0
69884,2023-01-01,20:00,2022-12-31,20:00,24,True,-4.0,257.0,1.0,0.0,0.0
69885,2023-01-01,21:00,2022-12-31,20:00,25,True,-5.0,257.0,4.0,0.0,30.0
69886,2023-01-01,22:00,2022-12-31,20:00,26,True,-5.0,257.0,3.0,0.0,20.0


In [71]:
raw_data = pd.read_parquet('../../data/weather_forecast_data/final_total_data/성산일출_prediction_weather_data.parquet')
raw_data[(raw_data['date']=='2021-06-29')&(raw_data['time']=='20:00')]

,date,forecast,풍향,3시간기온,강수확률,풍속,하늘상태,time
349512,2021-06-29,4,84.000000,24.000000,10.000000,3.700000,1.000000,20:00
349513,2021-06-29,7,76.000000,25.000000,20.000000,4.400000,3.000000,20:00
349514,2021-06-29,10,81.000000,25.000000,10.000000,4.400000,3.000000,20:00
349515,2021-06-29,13,90.000000,24.000000,0.000000,3.400000,1.000000,20:00
349516,2021-06-29,16,87.000000,23.000000,0.000000,1.700000,1.000000,20:00
349517,2021-06-29,19,98.000000,23.000000,0.000000,1.500000,1.000000,20:00
349518,2021-06-29,22,31.000000,23.000000,20.000000,1.700000,3.000000,20:00
349519,2021-06-29,25,21.000000,23.000000,20.000000,2.600000,3.000000,20:00
349520,2021-06-29,28,72.000000,25.000000,20.000000,2.800000,3.000000,20:00
349521,2021-06-29,31,61.000000,26.000000,10.000000,4.500000,3.000000,20:00


In [72]:
sample[-150:]

,date,time,forecast_date,forecast_time,hours_ahead,exact_match,3시간기온,풍향,하늘상태,풍속,강수확률
56586,2021-06-26,18:00,2021-06-25,20:00,22,True,22.0,152.0,4.0,4.0,30.0
56587,2021-06-26,19:00,2021-06-25,20:00,23,False,22.0,142.0,4.0,3.0,30.0
56588,2021-06-26,20:00,2021-06-25,20:00,24,False,22.0,131.0,4.0,3.0,30.0
56589,2021-06-26,21:00,2021-06-25,20:00,25,True,22.0,121.0,4.0,3.0,30.0
56590,2021-06-26,22:00,2021-06-25,20:00,26,False,22.0,121.0,4.0,3.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...
56731,2021-07-02,19:00,2021-07-01,20:00,23,False,<NA>,<NA>,nan,<NA>,30.0
56732,2021-07-02,20:00,2021-07-01,20:00,24,False,<NA>,<NA>,nan,<NA>,30.0
56733,2021-07-02,21:00,2021-07-01,20:00,25,True,<NA>,<NA>,nan,<NA>,30.0
56734,2021-07-02,22:00,2021-07-01,20:00,26,False,<NA>,<NA>,nan,<NA>,30.0
